<center>
<img src="logo.png" height="900"> 
</center>


#  Анализируем чеки

В этом задании мы будем работать с покупками и чеками. Смотреть за корреляциями в покупках довольно полезно.

> В 1992 году группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (Drug Store — формат разнокалиберных магазинов у дома). После анализа всех этих транзакций самым сильным правилом получилось «Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе». К сожалению, такое правило показалось руководству Osco Drug настолько контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали. Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с работы домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайший магазин. И мужья, не долго думая, совмещали приятное с полезным — покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения.

Для работы будем использовать датасет о продуктовых корзинах: https://www.kaggle.com/heeraldedhia/groceries-dataset

In [2]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')  # стиль для графиков
%matplotlib inline

Подружаем данные и смотрим как они выглядят.

In [4]:
df = pd.read_csv('data/groceries.csv', sep=',')
df.columns = ['id', 'fielddate', 'product']
print(df.shape)
df.head()

(38765, 3)


,id,fielddate,product
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


## 1. Корреляции

Для начала поработаем с корреляциями в данных. 

__а)__ Какой товар покупался чаще всего? Сохраните название этого товара в переменную `product_name`.

In [5]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: строка, пример: 'pip fruit'
# Вокруг ответа не должно быть никаких надписей array(['pip fruit']) или Index('pip fruit')
# Это должна быть именно строка, а не строка в массиве
product_name = df['product'].value_counts().index[0]

# your code here
print(product_name)

whole milk


In [6]:
# проверка, что задание решено корректно
assert len(product_name) == 10

# Аналогичные тесты скрыты от вас

__б)__ Сколько всего уникальных заказов было сделано? Сохраните число заказов в переменную `n_cnt`.

In [7]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: целое число, пример: 5555
n_cnt = df['id'].nunique()

# your code here
print(n_cnt)

3898


In [8]:
# проверка, что задание решено корректно
assert n_cnt > 3800
assert n_cnt < 4000

# Аналогичные тесты скрыты от вас

В таблице выше в каждой строке записана информация о покупке конкретного товара. Давайте сделаем табличку размера "число товаров" на "число покупок", чтобы понимать какие товары покупались вместе, а какие нет. 

> Обратите внимание, то здесь задание немного упрощено. Вообще говоря, нам нужно делать агрегацию по паре `fielddate, id`, если мы хотим изучать чеки по-честному. Но мы делаем её только по `id` для того, чтобы не усложнять задание. В качестве необязательного дополнения вы можете после сдачи задания переделать код так, чтобы дата тоже учитывалась при расчётах. 

In [9]:
sparse_sales = pd.pivot_table(df, 
               values='fielddate', 
               index='id', 
               columns='product', 
               fill_value=0, aggfunc='count')

sparse_sales.head()

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
id,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,1,0
1001,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,2,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0


В нашей матрице огромное число нулей. Обычно такие матрицы называют разряжеными. Мы занимаем нулями кучу свободной памяти, которую мы могли бы не занимать, если бы хранили данные [в ином виде.](https://cmdlinetips.com/2018/03/sparse-matrices-in-python-with-scipy/)

__в)__ Постройте матрицу корреляций Пирсона. Для этого используйте метод таблицы `.corr`.

In [11]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: таблица pd.DataFrame размера (167, 167)
sales_correlation = sparse_sales.corr()

# your code here
sales_correlation

product,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
product,,,,,,,,,,,,,,,,,,,,,
Instant food products,1.000000,-0.006936,-0.009420,-0.010825,-0.003470,-0.004007,-0.010419,-0.008275,0.005847,-0.014528,...,0.025355,0.021851,0.005348,0.014025,-0.005670,0.016640,0.002853,0.015981,0.005071,0.018221
UHT-milk,-0.006936,1.000000,0.013806,0.006105,-0.007877,0.018349,0.009462,-0.018785,0.002897,0.020801,...,-0.003803,-0.024466,0.009144,0.006618,-0.012871,-0.011550,0.009990,0.028747,0.006505,0.028753
abrasive cleaner,-0.009420,0.013806,1.000000,-0.006523,-0.002091,-0.002415,-0.013429,-0.004986,0.018970,0.001813,...,0.012922,-0.008507,-0.007390,0.002163,-0.003417,0.010777,0.000107,0.003558,0.008360,-0.009420
artif. sweetener,-0.010825,0.006105,-0.006523,1.000000,0.105251,-0.002775,-0.015432,-0.005730,-0.006645,0.016042,...,0.049392,-0.009776,0.010179,-0.006614,-0.003926,-0.006806,-0.018394,0.029591,-0.024397,-0.010825
baby cosmetics,-0.003470,-0.007877,-0.002091,0.105251,1.000000,-0.000889,-0.004947,-0.001837,0.014798,-0.007983,...,-0.003919,-0.003134,-0.007413,-0.011288,-0.001259,-0.008476,-0.005896,-0.021056,-0.000410,-0.003470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
white bread,0.016640,-0.011550,0.010777,-0.006806,-0.008476,-0.009789,-0.011195,0.005394,0.015850,0.030758,...,0.014921,0.001920,-0.000010,0.069757,0.004792,1.000000,0.046272,0.053497,0.030962,0.016640
white wine,0.002853,0.009990,0.000107,-0.018394,-0.005896,-0.006809,-0.010370,0.022577,0.015060,0.021670,...,-0.005083,0.028101,0.015072,0.000317,0.017039,0.046272,1.000000,0.032862,0.039600,-0.006953
whole milk,0.015981,0.028747,0.003558,0.029591,-0.021056,-0.024316,0.020192,0.032618,0.073856,0.028186,...,0.013874,-0.007158,0.069096,0.060338,0.039298,0.053497,0.032862,1.000000,0.103533,0.043075


Какие продукты сильнее всего коррелируют с яйцами, `domestic eggs` (их чаще всего покупают вместе)?  Сохраните название самого скоррелированного продукта в переменную `top_1`.

In [12]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: строка, пример: 'pip fruit'
# Вокруг ответа не должно быть никаких надписей array(['pip fruit']) или Index('pip fruit')
# Это должна быть именно строка, а не строка в массиве
top_1 = sales_correlation['domestic eggs'].sort_values(ascending = False).index[1]

# your code here
print(top_1)

meat spreads


Какие продукты "мешают" купить яйца, то есть отрицательно коррелируют с их покупкой? Сохраните название продукта с самой большой отрицательной корреляцией в переменную `bottom_1`.

In [14]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: строка, пример: 'pip fruit'
# Вокруг ответа не должно быть никаких надписей array(['pip fruit']) или Index('pip fruit')
# Это должна быть именно строка, а не строка в массиве
bottom_1 = sales_correlation['domestic eggs'].sort_values(ascending = True).index[0]

# your code here
print(bottom_1)

pet care


In [15]:
# проверка, что задание решено корректно
assert len(bottom_1) == 8
assert len(top_1) == 12

# Аналогичные тесты скрыты от вас

Напишите код, который выводит самые коррелируемые товары для случайного продукта из списка `unique_products`.

In [20]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

unique_products = df['product'].unique()

# your code here
sales_correlation[np.random.choice(unique_products)].sort_values(ascending = False).head(5)

product
mayonnaise         1.000000
tropical fruit     0.061490
flour              0.058050
coffee             0.046627
root vegetables    0.044955
Name: mayonnaise, dtype: float64

__г)__ Какие два продукта коррелируют сильнее всего? Положите их название в лист `answer`

In [24]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: массив из строк, пример: ['soups', 'apple', 'iphone 18']
answer = list(sales_correlation.unstack()[sales_correlation.unstack() != 1].sort_values(ascending = False).index[0])

# your code here
print(answer)

['soups', 'preservation products']


In [25]:
# проверка, что задание решено корректно
assert 'soups' in answer

# Аналогичные тесты скрыты от вас

Конечно же, корреляция — это [не единственный способ искать](https://habr.com/ru/company/ods/blog/353502/) между покупками ассоциативные правила.

## 2. Зависимость. 

В лекции мы с вами сказали, что события $A$ и $B$ называются независимыми, если $P(AB) = P(A)\cdot P(B)$. Отталкиваясь от этого определения, можно ввести другую характеристику, которая показывает, насколько продукты зависят друг от друга, а именно __поддержку (lift).__ 

$$
lift = \frac{P(AB)}{P(A)\cdot P(B)}
$$

Эта метрика описывает отношение зависимости товаров к их независимости. Если оказалось, что `lift = 1`, это означает, что покупка товара $A$ не зависит от покупки товара $B$. Если `lift > 1`, то это означает, что вероятность встретить оба товара в чеке, $P(AB)$ высокая, то есть товары покупают вместе. Если `lift < 1`, это означает, что товары, наоборот, очень часто покупают по-отдельности. 

__д)__ Посчитайте значение нашей метрики для яиц и молока (`'whole milk', 'domestic eggs'`). Запишите получившиеся значение метрики в переменную `answer`.

> Вам аккуратно нужно сделать три среза по условию  `>= 1`. Там, где пара надо делать срез так, чтобы оба товара дали `True`. Сделать это в одну строку вам поможет метод `.all(axis=1)`. Частоты можно получить методом `.mean()`, так как python думает, что `False` - это ноль, а `True` - это единица.

In [27]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: действительное чиссло, пример: 3.1418281828
answer = len(sparse_sales[(sparse_sales['whole milk'] >= 1) & (sparse_sales['domestic eggs'] >= 1)]) / (len(sparse_sales) * (sparse_sales['whole milk'] >= 1).mean() * (sparse_sales['domestic eggs'] >= 1).mean())

# your code here
print(answer)

1.152241691425711


In [28]:
# проверка, что задание решено корректно
assert answer < 3
assert answer > 1

# Аналогичные тесты скрыты от вас

__е)__ Посчитайте значение метрики для всех пар продуктов из датасета. Сохраните значения в словарик `dict`. В качестве ключа используете кортеж из пары продуктов. Чтобы удобнее было перебрать все сочетания, используйте `combinations` из модуля `itertools`.

Чтобы при подсчётах не возникало деления на ноль, добавьте к знаменателю маленькое число, например `1e-10`.

In [35]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here
from itertools import combinations

unique_products = sparse_sales.columns
dict = {}

for pair in combinations(unique_products, 2):
    prod1, prod2 = pair
    filtered_sales = sparse_sales[(sparse_sales[prod1] >= 1) & (sparse_sales[prod2] >= 1)]
    together = len(filtered_sales) / len(sparse_sales)
    
    prob1 = (sparse_sales[prod1] >= 1).mean()
    prob2 = (sparse_sales[prod2] >= 1).mean()
    
    lift = together / ((prob1 * prob2) + 1e-10)
    dict[pair] = lift
    
    print(lift)

0.8492374024855327
0.0
0.0
0.0
0.0
0.5369144881805007
0.0
1.254720961857163
0.6266880518641402
1.073829088731453
1.154496452498962
0.8579031351834197
0.0
0.9806288839620395
1.0542257741097623
0.7703556541166336
2.9198493564537316
0.0
0.6187300841171836
0.9079192189562852
2.824637059149614
0.0
0.0
0.7598439420528671
1.5468244641636575
0.7467431863379534
1.9887753817250995
2.12057353006503
0.0
0.988427812313113
0.0
2.6698620875122563
0.0
1.015104109286079
2.7942644720934897
0.0
3.821562934669496
0.0
0.5649274947645576
1.1034677396796368
1.412318063067709
0.0
1.9686853579292583
0.7711176472963746
0.0
2.6698620875122563
0.48847108493700486
1.9393027495883222
1.0014129248045196
0.0
0.0
2.2402279287972298
0.9150233109949077
3.8786054991766443
0.0
0.8484452335459589
0.0
1.443703297477571
0.6432341621524797
0.0
0.2651700406183624
0.9023144974506709
1.136916594688458
1.067214181328941
0.9107475557767422
0.0
0.5260458300070656
0.8302448742541211
2.4987176444989956
2.1230933305104815
0.0
0.0
0.95

2.167963488851137
0.0
0.0
0.0
0.0
0.0
0.0
1.21357352305335
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7974629272304851
0.0
0.0
0.0
0.0
4.753649728186043
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0697031763583364
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.7669984848483946
0.0
1.8949913080571945
1.175220767820418
1.450187286984849
1.7305511020603355
1.2490420179462358
1.469586159426264
1.6955192309176996
1.3980040211416844
1.372236063818569
1.0186521560192976
0.723929696353605
1.4643122683527148
1.0738291368902255
1.4506698551895902
1.4006466311031924
0.0
1.2080576616145025
0.9419553703907473
0.0
1.4811436187655813
1.0683504464645213
1.625082706692098
0.5369144881805007
0.9802590047272597
0.0
0.88259919154796
0.0
1.0067148478150234
1.7319823485609214
2.14765660427757
0.0
0.0
1.3072702841940527
1.573125539290932
0.7003232008445267
0.0
0.0
0.8603379035428755
1.775622912868277
0.44129959577398
0.968868360645142
1.4424568654585868
1.2414210116997846
1.61074329598586
2.0134293570137
0.0
1.1343265205714388


1.1680389575438328
1.4408969881799911
1.1921101491979877
0.5926826779331749
1.5538677305035475
1.1625698798750388
1.2310420594203724
0.9378887902565525
1.1349701264714296
1.1400200304691595
1.7174326178927521
0.0
1.1754873259441798
0.6628687180350726
1.5743131671975943
1.2594505208766824
1.2188232648712356
1.2851537386596383
1.259450688334809
1.1365549459972863
1.1697901603515188
0.0
1.1807349659704964
1.0495418431027792
1.2028461667485968
1.18948123133282
1.001835749661121
1.2155346331568966
1.2469808874947916
1.1165343129904408
1.1327376922821837
1.8891758586033827
0.997584722118312
1.1661579745163921
1.8891758586033827
1.1079456045685623
1.2851537815797418
1.535915428750859
1.1807349961612663
1.5058649192922668
1.0444225415742323
1.7738742020228877
1.1876399271281632
1.3028799521113903
1.1449549508458206
1.2917442540603783
1.0281230166797726
0.9801172878985825
1.018673345343214
0.5997383713157388
1.3993895694196707
1.144955078595168
1.259450108672253
1.1543812889065195
0.80734018419

0.6996445754005013
0.6316983059729945
1.4312989516946866
0.0
0.0
0.6440845462300021
1.9908067913732244
0.7423345785691965
3.8645056603152588
2.737358090150115
0.0
0.0
0.0
0.0
2.126104152886599
1.3073952730992457
1.824905717969224
1.202291173086152
0.0
2.36744500540796
1.05652469925768
0.5918613878988606
0.5298114894428696
0.0
0.5712749942345722
0.9625877932343768
1.6579590075791337
1.4465312401212658
0.0
1.3272039439539354
1.816496517368705
0.4424015180947021
0.0
0.0
1.0442243395689568
1.059622905941115
0.0
1.1392478773296868
2.0610703064542637
1.1376037168362692
2.0214344795635943
1.5148094567599444
0.9318667425493722
0.9336729941063392
2.265400673949893
0.0
0.0
0.8759549564822959
0.0
0.0
5.83969372804002
0.8476983480951709
1.7876627571091457
0.5474717932793876
0.9318670693701397
0.6820562043176094
0.0
1.3686790450750574
0.0
2.4605474386245185
0.8110694451780407
0.9954030867069295
1.4181090271771286
2.168205218104505
0.310622322737102
1.3352973069864231
2.189885771123083
0.86728216054

0.0
0.0
1.499230550193128
1.5386841665800963
3.1101051261650596
1.0387112020770404
1.2601291040203588
2.214770815220311
0.0
1.6241664610146553
0.0
3.04530888502707
0.0
0.0
1.9887747393304378
2.4362494216037818
1.2154438568709236
1.2465795064779857
0.0
6.0906213850249245
0.0
2.18988717311755
1.082777701610362
1.1073858856230532
1.0315379581668673
0.964851303708657
0.691134658675716
1.3369664247057778
2.4362476864167757
2.1709156374642995
1.8046283601735011
2.4362476864167757
1.0765752696829793
0.994387626623067
2.3768281672412375
1.01510376500269
0.0
1.188414524041156
2.7450696882095453
0.21370612254859012
0.0
4.429537806342391
2.49871673184258
0.0
1.7063228310965055
0.7165439175099676
2.320235996749815
1.8046283601735011
2.214770815220311
0.0
1.444099860123166
1.8740380052100871
0.9744996298264731
1.086802897242926
1.1312603292105587
0.0
0.9857658418420359
1.6997091146361414
1.227673559172895
0.9276745080513813
0.8120830762683678
0.5427456384475563
0.524030355709051
0.8722699408437434


...

0.0
0.0
33.0334729458191
0.0
0.0
0.0
0.0
5.861640742234957
0.0
0.0
0.0
0.0
0.0
25.986403434725137
0.0
0.0
0.0
0.0
0.0
48.72407458340782
2.8598647502611594
4.3359214814524245
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
3.1898487342184647
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
15.167225503131574
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
6.464328653130822
0.0
0.0
0.0
2.182528938285194
3.5339933185020667
0.0
15.468160702586166
4.0547828845350455
2.077823477010162
2.900294340160422
1.6378147775123333
0.0
0.0
0.30529444988434196
1.881272751697962
0.0
3.9775448534261804
0.0
3.0596528065305786
0.8783234188192781
1.277195154611932
6.327906469355947
4.218607781468511
0.6415403267310251
0.9374697661821264
0.0
0.0
1.1379913942898476
2.2453907224258702
1.1797815397676346
1.0058834884196939
0.0
0.0
1.0708786738636482
1.046723867323602
0.0
0.8093852969398726
1.2001225912824045
0.0
0.0
1.856189804673862
0.0
0.0
0.0
0.8981563675817811
0.0
0.0
0.9192432293067996
0.4645637730066357
0.0
0.0
0.0

1.428885365211858
1.231343803497965
1.1113554674617359
0.0
0.7144426162842789
1.1777599493783066
1.2675597708772917
1.1988107633433056
1.2265289841061573
0.5547437930313031
1.428885516804232
1.0881513134807363
1.2999636067474094
1.3376799903461716
1.0965866336412384
1.4904754496423591
0.7144426162842789
0.0
1.6765590713056022
0.8272493087954116
0.0
3.1435471031095523
0.709833478705784
1.603851017367015
1.7682458323213632
1.0493870210562533
1.2763016167638583
0.0
1.473538024325453
0.0
1.5894344683286088
1.397132584784055
0.714442715766759
1.1548527610436168
0.3112423956837164
1.56062679102953
1.0302788062706087
0.0
0.6224848102477528
1.164276916211263
1.5717737120531294
1.0675716645698765
0.9623106906360068
0.3833595021152719
0.654905830385553
1.3667599862631186
1.45676628218982
1.5496363951283427
0.9651244788757583
0.0
1.4288848346388032
0.0
1.4434660058730842
1.1008534945292885
0.6934297287935753
1.4969273666490173
2.328553832422526
2.1433278488528367
0.0
1.1559700357794185
1.20905697

1.2461338917568174
1.098750313128685
0.48278420029961494
1.2255292066282646
1.219276526075891
1.1778432717966085
1.0152668542370824
1.3908783536604137
1.573518962789679
1.327656613286627
0.5310625603654074
1.0376416597799267
1.1234017876498459
1.221444116404475
1.1943974509110953
1.1493147322579709
1.659570629971163
1.0897528751007728
1.0497750396241103
1.1091062480795146
1.1290498283628723
1.0178701005155204
1.5984142681318194
1.0902479762193873
1.4793166747880908
2.041264861585641
1.450868349596279
1.3235992964936911
2.006519723955537
1.2793510573909466
2.167964175883457
1.4288848346388032
0.0
1.2574192521196983
0.0
3.9294324745269904
0.0
1.216857343967627
1.2830806534620505
1.5717739528007213
1.0378552908849121
1.1888836897050674
0.0
2.9470754843988027
0.0
0.353207635313705
1.0478494148104918
0.7144426162842789
1.2135740786169456
0.6224847536067962
2.0065200727071804
1.197998600753744
1.5717732305581669
1.5562119784185822
2.3285533040328663
1.5717732305581669
0.8489124037150112
1.06

0.0
1.9320934484012784
0.8619125141376787
0.0
2.253178531103952
1.5108518233917445
1.6005224922770236
0.23559985748081344
0.0
1.026464574107736
0.6287096003800994
1.3100040139612665
1.0630018901634932
0.0
0.7858868560265647
4.1913910015389035
1.9779628122220654
0.9314217105080101
2.2862164994472565
1.15876751086106
0.7469817224532604
0.17835734289620897
0.9200629288160822
0.0
0.9959756782712972
3.725685624621922
1.2574187385248932
1.2347816806072691
1.3686196146791338
1.8401254451674367
0.5239246241838386
0.0
1.2267505314248837
1.0626077587851743
1.7647990186330806
1.7343709376477707
0.0
1.9344908288927183
1.0264646767839434
1.1742437107018115
1.1094875060910683
1.1975416836887205
0.0
0.0
0.0
1.0213944155522259
0.6448303573381888
0.5029676433252441
0.8413958201594723
1.5013962201756588
0.0
1.1629311622784897
0.731057722775745
1.1827908759990597
1.0487994525570248
0.41913971646690124
3.9775494786608756
1.1672930095133462
1.061860080056455
0.0
0.0
0.0
0.8938313785545728
0.294633375081039

1.3341192319856674
1.2921766125207734
1.1104104003673738
0.960999501120622
1.4247074440966339
0.0
0.0
1.3715685840898248
0.5230107614807887
0.0
0.0
0.0
0.0
0.0
0.7377265937040921
1.7232526001858441
0.0
0.49967943094050576
1.114542134529631
0.0
1.165437335976423
1.5629506064935255
0.9783758449514657
0.9748960052116055
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.7779661528716494
0.0
0.0
0.0
1.1753351747547494
0.0
1.0241719505601057
2.060252045227171
1.1904712506947923
1.6063626109970948
0.0
2.03976894314303
0.7778109167078388
4.409499736058807
0.0
3.7017990269107366
0.0
0.0
0.6582791240897325
0.0
0.0
0.0
0.6630096947993487
0.0
0.8666072834772403
0.5810970657509902
1.287133517734478
1.449845358742971
0.0
2.166759180010266
1.169867680440544
1.894070516070329
1.4731661986439009
0.0
0.0
0.9605501751887505
2.54971168574896
0.7955099574020474
1.9222364418206621
1.3852167376594193
0.0
1.2645392850833996
1.6187706181997228
1.2471604435005488
1.226080986336365
1.9887749962882526
0.6691461995463625
0.72

Сколько пар продуктов покупали вместе хотя бы раз? Запишите ответ в переменную `answer`.

In [36]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# Формат ответа: целое число, пример: 5555
answer = sum(1 for value in dict.values() if value > 0)

# your code here
print(answer)

9824


Для какой пары продуктов метрика $lift$ оказалась самой большой? 

In [37]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here
print(max(dict, key = dict.get))

('preservation products', 'soups')


Сколько раз эти продукты встретились в выборке? Как думаете адеватно ли делать выводы по такому объёму данных? 

In [38]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here
print(len(sparse_sales[(sparse_sales[max(dict, key = dict.get)[0]] >= 1) & (sparse_sales[max(dict, key = dict.get)[1]] >= 1)]))

1


Для какой пары продуктов метрика оказывается самой маленькой? 

In [39]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here
print(min(dict, key = dict.get))

('Instant food products', 'abrasive cleaner')


In [40]:
# проверка, что задание решено корректно

assert answer < 10000
assert answer > 9000

# Аналогичные тесты скрыты от вас

## 3. Неоцениваемые задания

Выше мы увидели, что некоторые продукты встречаются в выборке очень редко. Понятное дело, что по ним у нас не получится построить хорошее ассоциативное правило. Попробуйте повторить расчёт той же метрики, но с условием что продукт покупали больше 10 раз. Изучите самые покупаемые вместе продукты и самые непокупаемые вместе продукты. Насколько сильно список отличается от полученного в предыдущем задании? 

In [ ]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you

# your code here
product_purchase_counts = sparse_sales.sum()
popular_products = product_purchase_counts[product_purchase_counts > 10].index
filtered_sales_data = sparse_sales[popular_products]
lift_values = {}

for pair in combinations(popular_products, 2):
    product1, product2 = pair
    transactions_with_both = filtered_sales_data[(filtered_sales_data[product1] >= 1) & (filtered_sales_data[product2] >= 1)]
    
    joint_purchase_probability = len(transactions_with_both) / len(filtered_sales_data)
    individual_purchase_probability1 = (filtered_sales_data[product1] >= 1).mean()
    individual_purchase_probability2 = (filtered_sales_data[product2] >= 1).mean()
    lift = joint_purchase_probability / ((individual_purchase_probability1 * individual_purchase_probability2) + 1e-10)
    lift_values[pair] = lift

print(f'Pair with highest lift: {max(lift_values, key=lift_values.get)}')
print(f'Pair with lowest lift: {min(lift_values, key=lift_values.get)}')


Иногда в чеках пытаются искать __продукты-якоря.__ То есть продукты, которые являются основными. Например: айфон - основной продукт, наушники и чехол - дополнения к нему. Подумайте как можно попытаться найти такие продукты на основе простых метрик, основанных на подсчёте условных вероятностей.